# Linear Neural Network

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)
np.set_printoptions(precision=4)


# configuration
TEST_RATIO = 0.2

## Mathematical Representation

Let $\mathbf{x}^{[i]} = \{ x^{[i]}_1, x^{[i]}_2, \ldots x^{[i]}_{11}\}$ represent the $i^{th}$ feature vector, where all $\mathbf{x} \in \mathbb{R}^{11}$


And $y^{[i]}$ represent the true value of the $i^{th}$ feature vector, where $y \in \{0, 2, \ldots 10\}$


Given a training dataset $\mathcal{D}_{train}$ with size $n$ in the form

$$\mathcal{D}_{train} = \left\{ [\mathbf{x}^T, y]^{[1]}, \ldots, [\mathbf{x}^T, y]^{[n]} \right\}$$

#### A loss function, 

$$c : \mathbb{R}^{11} \times \mathbb{R} \times \mathbb{R}^{12} \mapsto \mathbb{R}$$

#### An empirical risk function 

$$\hat{\mathscr{l}}(\boldsymbol{\theta}) = \frac{1}{n} \sum_{i=1}^{n}c(\mathbf{x}^{[i]}, y^{[i]}, \boldsymbol{\theta})$$

Where $\boldsymbol{\theta}$ is the parameter vector of the form $\boldsymbol{\theta} = [\mathbf{w}^T, b]$

#### Goal: to find a $\boldsymbol{\theta}^*$ which minimizes the empirical risk

$$\boldsymbol{\theta}^* = \underset{\boldsymbol{\theta}}{argmin} \;\; \hat{\mathscr{l}}(\boldsymbol{\theta})$$


#### Linear Neural Network


$$\hat{y} = w_1x_1 + \ldots + w_{11}x_{11} + b$$


$$\hat{y}^{[i]} = \mathbf{w}^T \mathbf{x}^{[i]} + b$$

Let our loss function be the *squared error* function, then

$$c(\mathbf{x}^{[i]}, y^{[i]}, \boldsymbol{\theta}) = \frac{1}{2} \left( \hat{y}^{[i]} - y^{[i]} \right)^2$$


Thus the empirical risk function for this linear network is of the form

$$\hat{\mathscr{l}}(\boldsymbol{\theta}) = \frac{1}{n} \sum_{i=1}^{n} \frac{1}{2} \left( \mathbf{w}^T \mathbf{x}^{[i]} + b - y^{[i]} \right)^2$$

## Python Code


- $\mathcal{D}_{train}$ is `train`
- $\mathcal{D}_{test}$ is `test`
- $\boldsymbol{\theta}$ is `theta`
- $\mathbf{w}$ is `weights`
- $b$ is `bias`
- $\hat{\mathscr{l}}$ is `risk_fn`
- $\mathbf{x}$ is `x`
- $\hat{y}$ is `y_pred`
- $y$ is `y_true`
- $c$ is `loss_fn`

In [ ]:
# utility functions
class Utilities:
    @staticmethod
    def train_test_split(
        dataset: np.ndarray, test_ratio: float, shuffle: bool = True
    ) -> tuple[np.ndarray, np.ndarray]:

        if shuffle:
            np.random.shuffle(dataset)
        n = dataset.shape[0]
        n_test = int(test_ratio * n)
        return dataset[:n_test, :], dataset[n_test:, :]

    @staticmethod
    def xy_split(
        dataset: np.ndarray, output_last: bool = True
    ) -> tuple[np.ndarray, np.ndarray]:
        if output_last:
            return dataset[:, :-1], dataset[:, -1:]
        else:
            return dataset[:, 1:], dataset[:, :1]

## Data Preparation

In [ ]:
white_wine_csv = "data/raw/winequality-white.csv"
red_wine_csv = "data/raw/winequality-red.csv"

white_wine = pd.read_csv(white_wine_csv, delimiter=";")
white_wine_raw = white_wine.to_numpy()

## Linear Neural Network Class

In [ ]:
class LinearNN:
    def __init__(self, n_features: int, learning_rate: float) -> None:
        self.weights, self.bias = self.init_theta((n_features, 1), (1, 1))
        self.eta = learning_rate

    def forward(self, xs: np.ndarray) -> float:
        return np.double((self.weights.T @ xs.T) + self.bias).T

    def losses(self, y_trues: np.ndarray, y_preds: np.ndarray) -> np.ndarray:
        return np.square(y_trues - y_preds) * 0.5

    def risk(self, y_trues: np.ndarray, y_preds: np.ndarray) -> float:
        return np.mean(self.losses(y_trues, y_preds))

    def weight_gradient(
        self, xs: np.ndarray, y_trues: np.ndarray, y_preds: np.ndarray
    ) -> np.ndarray:
        return np.mean(xs * -(y_trues - y_preds), axis=0)

    def bias_gradient(self, y_trues: np.ndarray, y_preds: np.ndarray) -> np.ndarray:
        return np.double(np.mean(-(y_trues - y_preds)))

    def update_theta(self, xs: np.ndarray, y_trues: np.ndarray, y_preds: np.ndarray):
        self.weights -= self.eta * self.weight_gradient(xs, y_trues, y_preds).reshape(
            self.weights.shape
        )
        self.bias -= self.eta * self.bias_gradient(y_trues, y_preds)

    def train(self, xs: np.ndarray, ys: np.ndarray, rs: int = 10):
        for i in range(rs):
            y_preds = self.forward(xs)
            if (i + 1) % (10 ** (np.log10(rs) - 1)) == 0:
                print(f"Epoch {i + 1} :: risk={self.risk(ys, y_preds)}")
            self.update_theta(xs, ys, y_preds)

    def evaluate(self, x_test: np.ndarray, y_test: np.ndarray):
        print(f"Empirical Risk {self.risk(y_test, self.forward(x_test))}")

    @staticmethod
    def init_theta(
        weights_shape: tuple[int, int], bias_shape: tuple[int, int]
    ) -> tuple[np.ndarray, np.ndarray]:
        w, b = map(lambda x: x * 10, map(np.random.random, (weights_shape, bias_shape)))
        return w, b

In [ ]:
n = 2
true_ws = np.random.random(n) * 100
true_b = 0
xs = np.random.random((10000, n))

true_ys = (true_ws.T @ xs.T).reshape((xs.shape[0], 1))

test, train = Utilities.train_test_split(
    np.concatenate((xs, true_ys), axis=1), TEST_RATIO, shuffle=True
)
x_train, y_train = Utilities.xy_split(train)
x_test, y_test = Utilities.xy_split(test)

In [ ]:
# loss_fn=lambda y, yhat: 0.5 * np.sqrt(y - yhat)
model = LinearNN(x_train.shape[1], learning_rate=1)


model.train(x_train, y_train, rs=100)

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
model.weights.T, model.bias

In [ ]:
true_ws, true_b